# **차량 공유업체의 차량 파손 여부 분류하기**

## 0.미션

* 1) 미션1 : Data Preprocessing
    - **과제 수행 목표**
        - 본인의 구글 드라이브에 모델링 수행을 위해 적절한 폴더 및 파일로 **일관성 있게 정리**해야 합니다.
        - 제공된 데이터 : Car_Images.zip
            * Car_Images : 차량의 정상/파손 이미지 무작위 수집

* 2) 미션2 : CNN 모델링
    - **과제 수행 목표**
        - Tensorflow Keras를 이용하여 모델을 3개 이상 생성하세요.
            - 모델 구조와 파라미터는 자유롭게 구성하세요.
            - 단, 세부 목차에서 명시한 부분은 지켜주세요.

* 3) 미션3 : Data Argumentation & Transfer Learning
    - **과제 수행 목표**
        - 성능 개선을 위해 다음의 두가지를 시도하세요.
            * Data Augmentation을 적용하세요.(Image Generator)
            * Transfer Learning(VGG16)


## 1.환경설정

### (1) 데이터셋 폴더 생성
- **세부요구사항**
    - C드라이브에 Datasets라는 폴더를 만드세요.
        - 구글드라이브를 사용하는경우 드라이브 첫 화면에 Datasets 라는 폴더를 만드세요. ('/content/drive/MyDrive/Datasets/')
    - 해당 폴더 안에 Car_Images.zip 파일을 넣으세요.

* 구글 Colab을 이용하는 경우

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### (2) 데이터셋 불러오기
- **세부요구사항**
    - Car_Images.zip 파일을 C:/Datasets/ 경로에 압축 해제합니다.
    - zipfile 모듈을 이용하거나 다른 방식을 사용해도 됩니다.
        - 참고 자료 : [zipfile document](https://docs.python.org/3/library/zipfile.html#zipfile-objects)
    - 폴더구조(로컬)
        * C:/Datasets/ : 압축파일
        * C:/Datasets/Car_Images_train/ : 압축 해제한 이미지 저장소
    - 폴더구조(구글드라이브브)
        * /content/drive/MyDrive/Datasets/ : 압축파일
        * /content/drive/MyDrive/Datasets/Car_Images_train/ : 압축 해제한 이미지 저장소
    - 압축을 해제하면 다음과 같은 두 하위 폴더가 생성됩니다.
        * normal, abnormal : 각 폴더에는 이미지들이 있습니다.
        * 이후 단계에서 해당 경로로 부터 validation, test 셋을 추출하게 됩니다.
        

In [ ]:
import zipfile

In [ ]:
# 압축파일 경로
# 구글 드라이브인 경우 경로에 맞게 지정하세요.
dataset_path  = '/content/drive/MyDrive/Datasets/'
# dataset_path = 'C:/Datasets/'

file_path = dataset_path + 'Car_Images.zip'

In [ ]:
# 압축 해제
data = zipfile.ZipFile(file_path)
data.extractall('/content/drive/MyDrive/Datasets/Car_Images_train')

### (3) 이미지 저장을 위한 폴더 생성
- **세부요구사항**
    - train, validation, test 을 위해 각각 하위 폴더 normal과 abnormal를 준비합니다.
        - train
            * 정상 이미지 저장소 : C:/Datasets/Car_Images_train/normal/
                * 구글드라이브 :   /content/drive/MyDrive/Datasets/Car_Images_train/normal/
            * 파손 이미지 저장소 : C:/Datasets/Car_Images_train/abnormal/
                * 구글드라이브 : /content/drive/MyDrive/Datasets/Car_Images_train/abnormal/
        - val, test 역시 동일한 구조로 생성합니다.
    - 직접 탐색기에서 폴더를 생성할 수도 있고, os 모듈을 이용하여 코드로 작성할 수도 있습니다.
        - 참고 자료 : [os document](https://docs.python.org/3/library/os.html)

In [ ]:

# 각각 경로 지정
 os.mkdir("/content/drive/MyDrive/Datasets/Car_Images_train/normal")
 os.mkdir("/content/drive/MyDrive/Datasets/Car_Images_train/abnormal")

# train 폴더는 압축을 해제하면서 이미 생성 되어 있습니다.

# test 폴더 만들기 os.mkdir()
# os.mkdir("/content/drive/MyDrive/Datasets/Car_Images_test")
# os.mkdir("/content/drive/MyDrive/Datasets/Car_Images_test/normal")
# os.mkdir("/content/drive/MyDrive/Datasets/Car_Images_test/abnormal")


# validation 폴더 만들기
try:
  os.mkdir("/content/drive/MyDrive/Datasets/Car_Images_val")
  os.mkdir("/content/drive/MyDrive/Datasets/Car_Images_val/normal")
  os.mkdir("/content/drive/MyDrive/Datasets/Car_Images_val/abnormal")
except:
  pass

# try, except
# try:            # 실행하고싶은코드
#  ~~~
# except:         # try에서 오류가 났을 경우 대신 실행할 코드
#   pass -->오류뜨면 넘어감.

IndentationError: ignored

## 2.데이터 전처리

### (1) 데이터 분할 : Training set | Validation set | Test set 생성
- **세부요구사항**
    - Training set, Validation set, Test set을 만듭니다.
        * size
            * test : 전체에서 20%를 추출합니다.
            * validation : test를 떼어낸 나머지에서 다시 20%를 추출합니다.
        * 데이터는 랜덤하게 추출해야 합니다.
            - random, shutil 모듈을 이용하여 랜덤하게 추출할 수 있습니다.
                - [random document](https://docs.python.org/3/library/random.html) | [shutil document](https://docs.python.org/3/library/shutil.html)
            * 해당 모듈 이외에 자신이 잘 알고 있는 방법을 사용해도 됩니다.
---

#### 1) test, validation 크기를 지정

In [ ]:
import random, shutil
import os

In [ ]:
tr_n_path = '/content/drive/MyDrive/Datasets/Car_Images_train/normal'
tr_ab_path = '/content/drive/MyDrive/Datasets/Car_Images_train/abnormal'

In [ ]:
# 전체 이미지 갯수를 확인합니다.
len(os.listdir(tr_n_path)) , len(os.listdir(tr_ab_path))

(194, 194)

In [ ]:
# test 사이즈 : 전체 이미지의 20%
te_data_num = [round(len(os.listdir(tr_n_path))*0.2), round(len(os.listdir(tr_ab_path))*0.2)]
print(te_data_num)

# validation 사이즈 : test를 제외한 나머지 중에서 20%
val_data_num = [ round((len(os.listdir(tr_n_path))-te_data_num[0])*0.2) , round((len(os.listdir(tr_n_path))-te_data_num[1])*0.2) ]
print(val_data_num)

# train 사이즈
train_data_num = [len(os.listdir(tr_n_path)) - te_data_num[0] - val_data_num[0],
                  len(os.listdir(tr_ab_path))- te_data_num[1] - val_data_num[1]]

[60, 61]
[48, 48]


In [ ]:
 # os.listdir(tr_n_path)[0]
os.listdir(tr_n_path)[0]

'DALLíñE 2023-03-10 18.50.29 - photo of a part of car.png'

#### 2) test 셋 추출

In [ ]:
random.seed(2023)

normal_images = os.listdir('/content/drive/MyDrive/Datasets/Car_Images_train/normal')
abnormal_images = os.listdir('/content/drive/MyDrive/Datasets/Car_Images_train/abnormal')

for i in range (te_data_num[0]):
 shutil.move('/content/drive/MyDrive/Datasets/Car_Images_train/normal/'+os.listdir(tr_n_path)[i],'/content/drive/MyDrive/Datasets/Car_Images_test/normal' )

for i in range (te_data_num[1]):
 shutil.move('/content/drive/MyDrive/Datasets/Car_Images_train/abnormal/'+os.listdir(tr_ab_path)[i],'/content/drive/MyDrive/Datasets/Car_Images_test/abnormal' )


In [ ]:
train_n_path = '/content/drive/MyDrive/Datasets/Car_Images_train/normal'
train_ab_path = '/content/drive/MyDrive/Datasets/Car_Images_train/abnormal'
val_n_path = '/content/drive/MyDrive/Datasets/Car_Images_val/normal'
val_ab_path = '/content/drive/MyDrive/Datasets/Car_Images_val/abnormal'
test_n_path = '/content/drive/MyDrive/Datasets/Car_Images_test/normal'
test_ab_path = '/content/drive/MyDrive/Datasets/Car_Images_test/abnormal'

In [ ]:
# 추출 후 이미지 갯수 확인

# len(os.listdir('/content/drive/MyDrive/Datasets/Car_Images_test/normal'))
# len(os.listdir('/content/drive/MyDrive/Datasets/Car_Images_test/abnormal'))
# len(os.listdir('/content/drive/MyDrive/Datasets/Car_Images_train/abnormal'))

242

#### 3) validation 셋 추출

In [ ]:
random.seed(2023)

normal_images = os.listdir('/content/drive/MyDrive/Datasets/Car_Images_train/normal')
abnormal_images = os.listdir('/content/drive/MyDrive/Datasets/Car_Images_train/abnormal')

for i in range (val_data_num[0]):
 shutil.move('/content/drive/MyDrive/Datasets/Car_Images_train/normal/'+os.listdir(tr_n_path)[i],'/content/drive/MyDrive/Datasets/Car_Images_val/normal' )

for i in range (val_data_num[1]):
 shutil.move('/content/drive/MyDrive/Datasets/Car_Images_train/abnormal/'+os.listdir(tr_ab_path)[i],'/content/drive/MyDrive/Datasets/Car_Images_val/abnormal' )


In [ ]:
# 추출 후 이미지 갯수 확인

len(os.listdir('/content/drive/MyDrive/Datasets/Car_Images_val/normal'))
#len(os.listdir('/content/drive/MyDrive/Datasets/Car_Images_val/abnormal'))

48

In [ ]:
print(len(os.listdir(dataset_path+'Car_Images_test/normal')))
print(len(os.listdir(dataset_path+'Car_Images_test/abnormal')))
print(len(os.listdir(dataset_path+'Car_Images_train/normal')))
print(len(os.listdir(dataset_path+'Car_Images_train/abnormal')))
print(len(os.listdir(dataset_path+'Car_Images_val/normal')))
print(len(os.listdir(dataset_path+'Car_Images_val/abnormal')))

60
61
194
194
48
48


### (2) 데이터 복사 및 이동
- **세부요구사항**
    - 분할된 데이터를 복사 이동합니다.
        - 새로운 폴더에 저장하는 데이터로 "3.모델링I"에서 사용합니다.
        - 기존 폴더는 "4.모델링II > (1) Data Augmentation"에서 사용합니다.
    - Training set | Validation set | Test set의 데이터를 **새로운 폴더**에 복사하세요.
        - 새로운 폴더 명
            * copy_images/trainset
            * copy_images/validset
            * copy_images/testset
        - 새로운 폴더에는 normal, abnormal 파일 모두를 복사합니다.
            * 파일을 구분하기 위해 abnormal 파일들은 파일명 앞에 접두사 'ab_'를 붙입시다.
        - os, shutil 모듈을 활용하세요.

#### 1) abnormal 파일 복사

* 복사하기 : shutil.copytree()

In [ ]:
shutil.copytree('/content/drive/MyDrive/Datasets/Car_Images_train/abnormal', '/content/drive/MyDrive/Datasets/copy_images/trainset' )
shutil.copytree('/content/drive/MyDrive/Datasets/Car_Images_val/abnormal', '/content/drive/MyDrive/Datasets/copy_images/validset' )
shutil.copytree('/content/drive/MyDrive/Datasets/Car_Images_test/abnormal', '/content/drive/MyDrive/Datasets/copy_images/testset' )


'/content/drive/MyDrive/Datasets/copy_images/testset'

In [ ]:
print(len(os.listdir(dataset_path+'Car_Images_train/abnormal')))
print(len(os.listdir(dataset_path+'Car_Images_val/abnormal')))
print(len(os.listdir(dataset_path+'Car_Images_test/abnormal')))
print(len(os.listdir(dataset_path+'copy_images/trainset')))
print(len(os.listdir(dataset_path+'copy_images/validset')))
print(len(os.listdir(dataset_path+'copy_images/testset')))

194
48
61
383
96
121


* abnormal 이미지 이름의 접두어 "ab_" 붙이기 : os.rename

In [ ]:
copy_train_path='/content/drive/MyDrive/Datasets/copy_images/trainset/'
copy_val_path='/content/drive/MyDrive/Datasets/copy_images/validset/'
copy_test_path='/content/drive/MyDrive/Datasets/copy_images/testset/'

In [ ]:
for filename in os.listdir(copy_train_path):
  rename='ab_'+filename
  os.rename( copy_train_path+filename, copy_train_path+rename)
  filename=rename
for filename in os.listdir(copy_val_path):
  rename='ab_'+filename
  os.rename( copy_val_path+filename, copy_val_path+rename)
  filename=rename
for filename in os.listdir(copy_test_path):
  rename='ab_'+filename
  os.rename( copy_test_path+filename, copy_test_path+rename)
  filename=rename

#### 2) normal 파일 복사

In [ ]:
from distutils.dir_util import copy_tree

In [ ]:
copy_tree('/content/drive/MyDrive/Datasets/Car_Images_train/normal', copy_train_path)
copy_tree('/content/drive/MyDrive/Datasets/Car_Images_val/normal', copy_val_path)
copy_tree('/content/drive/MyDrive/Datasets/Car_Images_test/normal', copy_test_path)

* 데이터 갯수 조회

In [ ]:
print(len(os.listdir(dataset_path+'copy_images/trainset/')))
print(len(os.listdir(dataset_path+'copy_images/validset/')))
print(len(os.listdir(dataset_path+'copy_images/testset/')))

383
96
121


## 3.모델링 I
* **세부요구사항**
    * 모델링을 위한 데이터 구조 만들기
        * x : 이미지를 array로 변환합니다.
        * y : 이미지 갯수만큼 normal - 0, abnormal - 1 로 array를 만듭니다.
    * 모델을 최소 3개 이상 만들고 성능을 비교합니다.
        * 모델 학습 과정에 알맞은 보조 지표를 사용하세요.
        * 전처리 과정에서 생성한 Validation set을 적절하게 사용하세요.
        * Early Stopping을 반드시 사용하세요.
            * 최적의 가중치를 모델에 적용하세요.

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report, confusion_matrix

### (1) X : image to array
- **세부요구사항**
    * 모델링을 위해서는 np.array 형태로 데이터셋을 만들어야 합니다.
    * Training set / Validation set / Test set의 X는 이미지 형태로 되어있습니다.
    * 이미지 파일을 불러와 train, valid, test 각각 array 형태로 변환해 봅시다.
        * 각 폴더로 부터 이미지 목록을 만들고
        * 이미지 한장씩 적절한 크기로 로딩하여 (keras.utils.load_img)
            * 이미지가 너무 크면 학습시간이 많이 걸리고, 메모리 부족현상이 발생될 수 있습니다.
            * 이미지 크기를 280 * 280 * 3 이내의 크기를 설정하여 로딩하시오.
            * array로 변환 (keras.utils.img_to_array, np.expand_dims)
        * 데이터셋에 추가합니다.(데이터셋도 array)

#### 1) 이미지 목록 만들기
* train, validation, test 폴더로 부터 이미지 목록을 생성합니다.

In [ ]:
# 이미지 목록 저장
img_train_list = os.listdir(dataset_path+'copy_images/trainset/')
img_valid_list = os.listdir(dataset_path+'copy_images/validset/')
img_test_list = os.listdir(dataset_path+'copy_images/testset/')

In [ ]:
# 메모리, 처리시간을 위해서 이미지 크기 조정
img_size = 280 ## 사이즈 조정 가능

In [ ]:
len(img_train_list)
#len(img_valid_list)
#len(img_test_list)

383

In [ ]:
388+96+121

605

In [ ]:
img_train_list

#### 2) 이미지들을 배열 데이터셋으로 만들기

In [ ]:
from keras.preprocessing import image
from keras.utils import load_img, img_to_array

In [ ]:
x_train=[]
x_val=[]
x_test=[]
train_path = dataset_path+'copy_images/trainset/'
val_path = dataset_path + 'copy_images/validset/'
test_path = dataset_path + 'copy_images/testset/'

In [ ]:
# x_train
for file in img_train_list:
  img_path = train_path + file
  img = load_img(img_path, target_size=(280,280))
  img_array = img_to_array(img)
  x_train.append(img_array)
x_train = np.array(x_train)

# x_val
for file in img_valid_list:
  img_path = val_path + file
  img = load_img(img_path, target_size=(280,280))
  img_array = img_to_array(img)
  x_val.append(img_array)
x_val = np.array(x_val)

# x_test
for file in img_test_list:
  img_path = test_path + file
  img = load_img(img_path, target_size=(280,280))
  img_array = img_to_array(img)
  x_test.append(img_array)
x_test = np.array(x_test)

In [ ]:
print('x_train은',x_train.shape)
print('x_val은',x_val.shape)
print('x_test은',x_test.shape)

x_train은 (383, 280, 280, 3)
x_val은 (96, 280, 280, 3)
x_test은 (121, 280, 280, 3)


### (2) y : 클래스 만들기
- **세부요구사항**
    - Training set / Validation set / Test set의 y를 생성합니다.
        - 각각 normal, abnormal 데이터의 갯수를 다시 확인하고
        - normal을 0, abnormal을 1로 지정합니다.

In [ ]:
# 데이터 갯수 확인
print( len(img_train_list) )
print( len([val for val in img_train_list if val.startswith('ab_')]) )
print('---')
print( len(img_valid_list) )
print( len([val for val in img_valid_list if val.startswith('ab_')]) )
print('---')
print( len(img_test_list) )
print( len([val for val in img_test_list if val.startswith('ab_')]) )

383
194
---
96
48
---
121
61


* y_train, y_valid, y_test 만들기
    * normal, abnormal 데이터의 갯수를 다시 확인하고 normal을 0, abnormal을 1로 지정합니다.

In [ ]:
y_train=[]
y_val=[]
y_test=[]

for file in img_train_list:
 if file.startswith('ab_'):
  y_train.append(1)
 else:
  y_train.append(0)

for file in img_valid_list:
 if file.startswith('ab_'):
  y_val.append(1)
 else:
  y_val.append(0)

for file in img_test_list:
 if file.startswith('ab_'):
  y_test.append(1)
 else:
  y_test.append(0)

In [ ]:
y_train

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


In [ ]:
y_val

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.

In [ ]:
y_train=np.array(y_train)
y_val=np.array(y_val)
y_test=np.array(y_test)

In [ ]:
# y_train=np.expand_dims(y_train, axis=1)
# y_val=np.expand_dims(y_val, axis=1)
# y_test=np.expand_dims(y_test, axis=1)

In [ ]:
# y_train.shape
# y_val.shape
y_test.shape

(121,)

In [ ]:
x_train

1. min-max scaling

In [ ]:
max_v, min_v = x_train.max(), x_train.min()

In [ ]:
max_v, min_v

(255.0, 0.0)

In [ ]:
x_train = (x_train-min_v) / (max_v-min_v)
x_val = (x_val-min_v) / (max_v-min_v)
x_test = (x_test-min_v) / (max_v-min_v)

In [ ]:
x_train.max(), x_val.max(), x_test.max(), x_train.min(), x_val.min(), x_test.min()

(1.0, 1.0, 1.0, 0.0, 0.0, 0.0)

One-hot Encoding

In [ ]:
class_n = len(np.unique(y_train))

In [ ]:
from tensorflow.keras.utils import to_categorical

In [ ]:
class_n

2

In [ ]:
y_train = to_categorical(y_train, class_n)
y_val = to_categorical(y_val, class_n)
y_test = to_categorical(y_test, class_n)

In [ ]:
x_train.shape, y_train.shape

((383, 280, 280, 3), (383, 2))

### (3) 모델1
- **세부요구사항**
    - Conv2D, MaxPooling2D, Flatten, Dense 레이어들을 이용하여 모델을 설계
    - 학습시 validation_data로 validation set을 사용하시오.
    - 반드시 Early Stopping 적용
    - 평가시, confusion matrix, accuracy, recall, precision, f1 score 등을 이용하시오.

#### 1) 구조 설계

In [ ]:
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.backend import clear_session
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten, Conv2D, MaxPool2D, BatchNormalization, Dropout, GlobalAveragePooling2D

from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
## Functional API

# 1번 세션 클리어
clear_session()
# 2번 레이어 엮기
il = Input( shape=(280,280,3) )

hl = Conv2D(filters=32, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu')(il)
hl = Conv2D(filters=32, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu')(hl)
hl = MaxPool2D(pool_size=(2,2))(hl)
hl = BatchNormalization()(hl)

hl = Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu')(hl)
hl = Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu')(hl)
hl = MaxPool2D(pool_size=(2,2))(hl)
hl = BatchNormalization()(hl)

hl = Conv2D(filters=128, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu')(hl)
hl = Conv2D(filters=128, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu')(hl)
hl = MaxPool2D(pool_size=(2,2))(hl)
hl = BatchNormalization()(hl)

hl = Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu')(hl)
hl = Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu')(hl)
hl = MaxPool2D(pool_size=(2,2))(hl)
hl = BatchNormalization()(hl)

hl = Conv2D(filters=512, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu')(hl)
hl = Conv2D(filters=512, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu')(hl)
hl = MaxPool2D(pool_size=(2,2))(hl)
hl = BatchNormalization()(hl)

hl = GlobalAveragePooling2D()(hl)
hl = Dropout(0,1)(hl)
ol= Dense(2, activation='sigmoid')(hl)

# 3번 모델의 시작과 끝 지정
model1 = Model(il, ol)

# 4번 컴파일
model1.compile(loss=keras.losses.binary_crossentropy, metrics=['accuracy'], optimizer='adam')
# 요약
model1.summary()

NameError: ignored

#### 2) 학습
* EarlyStopping 설정하고 학습시키기

In [ ]:
es = EarlyStopping(monitor='val_loss',
                   min_delta=0,
                   patience=5,
                   verbose=1,
                   restore_best_weights=True)

In [ ]:
hist = model1.fit(x_train, y_train, validation_data=(x_val, y_val),
                 batch_size=128, epochs=10000, callbacks=[es], verbose=1)

Epoch 1/10000
3/3 [==============================] - 52s 11s/step - loss: 1.1177 - accuracy: 0.5666 - val_loss: 0.7576 - val_accuracy: 0.5000
Epoch 2/10000
3/3 [==============================] - 3s 1s/step - loss: 0.7194 - accuracy: 0.8329 - val_loss: 1.4195 - val_accuracy: 0.4792
Epoch 3/10000
3/3 [==============================] - 3s 1s/step - loss: 0.4758 - accuracy: 0.8355 - val_loss: 1.7234 - val_accuracy: 0.4896
Epoch 4/10000
3/3 [==============================] - 3s 1s/step - loss: 0.4001 - accuracy: 0.8486 - val_loss: 0.7230 - val_accuracy: 0.5208
Epoch 5/10000
3/3 [==============================] - 3s 1s/step - loss: 0.3177 - accuracy: 0.8799 - val_loss: 1.1773 - val_accuracy: 0.5000
Epoch 6/10000
3/3 [==============================] - 3s 1s/step - loss: 0.3003 - accuracy: 0.8930 - val_loss: 1.5013 - val_accuracy: 0.5000
Epoch 7/10000
3/3 [==============================] - 3s 1s/step - loss: 0.2632 - accuracy: 0.8956 - val_loss: 1.4709 - val_accuracy: 0.4688
Epoch 8/10000
3/3 

In [ ]:
x_val.shape, x_train.shape, y_val.shape, y_train.shape,

((96, 280, 280, 3), (383, 280, 280, 3), (96, 2), (383, 2))

In [ ]:
y_pred = model1.predict(x_test)
y_pred, y_test

In [ ]:
# model1_y_pred = np.where(y_pred>0.6,1,0)
# model1_y_test = np.where(y_test>0.6,1,0)
model1_y_test = y_test.argmax(axis=1)
model1_y_pred = y_pred.argmax(axis=1)

#### 3) test set으로 예측하고 평가하기
* 평가는 confusion_matrix, classification_report 활용

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

In [ ]:
model1_y_pred[:5], model1_y_test[:5]

(array([0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0]))

In [ ]:
model1_acc = accuracy_score(model1_y_test, model1_y_pred)
print(f'테스트셋 정확도 : {model1_acc*100}%')

테스트셋 정확도 : 54.54545454545454%


In [ ]:
print(classification_report(model1_y_test, model1_y_pred))

              precision    recall  f1-score   support

           0       0.52      1.00      0.69        60
           1       1.00      0.10      0.18        61

    accuracy                           0.55       121
   macro avg       0.76      0.55      0.43       121
weighted avg       0.76      0.55      0.43       121



### (4) 모델2
- **세부요구사항**
    - Conv2D, MaxPooling2D, Flatten, Dense 레이어들을 이용하여 모델을 설계
    - 학습시 validation_data로 validation set을 사용하시오.
    - 반드시 Early Stopping 적용
    - 평가시, confusion matrix, accuracy, recall, precision, f1 score 등을 이용하시오.

#### 1) 구조 설계

In [ ]:
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense

In [ ]:
keras.backend.clear_session()

base_model2 = tf.keras.applications.EfficientNetV2B3(
              include_top=False,
              weights="imagenet",
              input_shape=None)

new_output = GlobalAveragePooling2D()(base_model2.output)
new_output = Dense(2, activation = 'sigmoid')(new_output)

model2 = keras.models.Model(base_model2.inputs, new_output)

model2.summary()

In [ ]:
print(f'모델의 레이어 수 : {len(model2.layers)}')

모델의 레이어 수 : 411


In [ ]:
for idx, layer in enumerate(model2.layers) :
    if idx < 311 :
        layer.trainable = False # 레이어의 학습을 안시킴. FROZEN.
    else :
        layer.trainable = True

In [ ]:
model2.compile(loss='binary_crossentropy', metrics=['accuracy'],
             optimizer=keras.optimizers.Adam(learning_rate=0.001) )

#### 2) 학습
* EarlyStopping 설정하고 학습시키기

In [ ]:
es = EarlyStopping(monitor='val_loss',
                   min_delta=0,
                   patience=5,
                   verbose=1,
                   restore_best_weights=True)

In [ ]:
hist = model2.fit(x_train, y_train, validation_data=(x_val, y_val),
                 batch_size=256, epochs=1000, callbacks=[es], verbose=1)

#hist = model.fit(datagen.flow(train_x, train_y),
                #  epochs=1000, validation_data=(valid_x, valid_y),
                #  verbose=1, callbacks=[es, lr_reduction] )

Epoch 1/1000
2/2 [==============================] - 13s 11s/step - loss: 0.7511 - accuracy: 0.4360 - val_loss: 0.6912 - val_accuracy: 0.5729
Epoch 2/1000
2/2 [==============================] - 3s 2s/step - loss: 0.7107 - accuracy: 0.5039 - val_loss: 0.6983 - val_accuracy: 0.5000
Epoch 3/1000
2/2 [==============================] - 3s 1s/step - loss: 0.7055 - accuracy: 0.5065 - val_loss: 0.7482 - val_accuracy: 0.5000
Epoch 4/1000
2/2 [==============================] - 3s 1s/step - loss: 0.7044 - accuracy: 0.4883 - val_loss: 0.7520 - val_accuracy: 0.5000
Epoch 5/1000
2/2 [==============================] - 3s 1s/step - loss: 0.7024 - accuracy: 0.5091 - val_loss: 0.7260 - val_accuracy: 0.5000
Epoch 6/1000
2/2 [==============================] - 4s 2s/step - loss: 0.6932 - accuracy: 0.5248 - val_loss: 0.7110 - val_accuracy: 0.5000
Epoch 6: early stopping


#### 3) test set으로 예측하고 평가하기
* 평가는 confusion_matrix, classification_report 활용

### (5) 모델3
- **세부요구사항**
    - Conv2D, MaxPooling2D, Flatten, Dense 레이어들을 이용하여 모델을 설계
    - 학습시 validation_data로 validation set을 사용하시오.
    - 반드시 Early Stopping 적용
    - 평가시, confusion matrix, accuracy, recall, precision, f1 score 등을 이용하시오.

#### 1) 구조 설계

In [ ]:
## Functional API

# 1번 세션 클리어
keras.backend.clear_session()
# 2번 레이어 엮기
il = Input( shape=(280,280,3) )

hl = Conv2D(filters=32, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu')(il)
hl = Conv2D(filters=32, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu')(hl)
hl = Conv2D(filters=32, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu')(hl)
hl = MaxPool2D(pool_size=(2,2))(hl)
hl = BatchNormalization()(hl)

hl = Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu')(il)
hl = Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu')(hl)
hl = Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu')(hl)
hl = MaxPool2D(pool_size=(2,2))(hl)
hl = BatchNormalization()(hl)

hl = Conv2D(filters=128, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu')(il)
hl = Conv2D(filters=128, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu')(hl)
hl = Conv2D(filters=128, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu')(hl)
hl = MaxPool2D(pool_size=(2,2))(hl)
hl = BatchNormalization()(hl)

hl = Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu')(hl)
hl = Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu')(hl)
hl = Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu')(hl)
hl = MaxPool2D(pool_size=(2,2))(hl)
hl = BatchNormalization()(hl)

hl = Conv2D(filters=512, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu')(hl)
hl = Conv2D(filters=512, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu')(hl)
hl = Conv2D(filters=512, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu')(hl)
hl = MaxPool2D(pool_size=(2,2))(hl)
hl = BatchNormalization()(hl)

hl = Flatten()(hl)
hl = Dropout(0,25)(hl)
ol= Dense(2, activation='sigmoid')(hl)

# 3번 모델의 시작과 끝 지정
model3 = Model(il, ol)

# 4번 컴파일
model3.compile(loss=keras.losses.binary_crossentropy, metrics=['accuracy'], optimizer='adam')
# 요약
model3.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 280, 280, 3)]     0         
                                                                 
 conv2d_6 (Conv2D)           (None, 280, 280, 128)     3584      
                                                                 
 conv2d_7 (Conv2D)           (None, 280, 280, 128)     147584    
                                                                 
 conv2d_8 (Conv2D)           (None, 280, 280, 128)     147584    
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 140, 140, 128)    0         
 2D)                                                             
                                                                 
 batch_normalization_2 (Batc  (None, 140, 140, 128)    512       
 hNormalization)                                             

#### 2) 학습
* EarlyStopping 설정하고 학습시키기

In [ ]:
es = EarlyStopping(monitor='val_loss',
                   min_delta=0,
                   patience=8,
                   verbose=1,
                   restore_best_weights=True)

In [ ]:
hist = model3.fit(x_train, y_train, validation_data=(x_val, y_val),
                 batch_size=256, epochs=1000, callbacks=[es], verbose=1)

#### 3) test set으로 예측하고 평가하기
* 평가는 confusion_matrix, classification_report 활용

## 4.모델링 II
* **세부요구사항**
    - 성능을 높이기 위해서 다음의 두가지를 시도해 봅시다.
        - Data Augmentation을 통해 데이터를 증가 시킵니다.
            - ImageDataGenerator를 사용합니다.
        - 사전 학습된 모델(Transfer Learning)을 가져다 사용해 봅시다.
            - VGG16(이미지넷)을 사용해 봅시다.

### (1) Data Augmentation
- **세부요구사항**
    * 모델 학습에 이용할 이미지 데이터를 증강시키세요.
    * Keras의 ImageDataGenerator를 이용
        - [ImageDataGenerator document](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator)

    * image generator를 이용하여 학습
        * 모델 구조는 이미 생성한 1,2,3 중 하나를 선택하여 학습


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
img_size = 280 ## 사이즈 조정 가능

train_path = dataset_path+'Car_Images_train/'
valid_path = dataset_path+'Car_Images_val/'

#### 1) ImageGenerator 생성
* ImageDataGenerator 함수 사용
    * 주요 옵션
        * rotation_range: 무작위 회전을 적용할 각도 범위
        * zoom_range: 무작위 줌을 적용할 범위 [1-zoom_range, 1+zoom_range]
        * horizontal_flip: 무작위 좌우반전을 적용할지 여부
        * vertical_flip: 무작위 상하반전을 적용할지 여부
        * rescale: 텐서의 모든 값을 rescale 값으로 나누어줌 (이 경우에는 255로 나누어서 0~1사이의 값으로 변경)

In [ ]:
train_datagen = ImageDataGenerator(rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   zoom_range=0.3,
                                   shear_range=0.1,
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   fill_mode='nearest')

valid_datagen = ImageDataGenerator(rotation_range=30,
                                   zoom_range=0.3)

# shear_range=이미지 굴절, zoom_range=확대축소, rotation_range=회전범위

In [ ]:
# train_datagen.fit(x_train)
# valid_datagen.fit(x_val)

In [ ]:
!mkdir /content/drive/MyDrive/Datasets/Car_Images_train/gen
!mkdir /content/drive/MyDrive/Datasets/Car_Images_val/gen

mkdir: cannot create directory ‘/content/drive/MyDrive/Datasets/Car_Images_val/gen’: File exists


In [ ]:
train_datagen

#### 2) 경로로 부터 이미지 불러 올 준비
* .flow_from_directory 이용
    * 디렉토리에서 이미지를 가져와서 데이터 증강을 적용하고 batch 단위로 제공하는 generator를 생성합니다.
    * 이미지를 불러올 때 target_size로 크기를 맞추고,
    * class_mode로 이진 분류(binary)를 수행하도록 지정합니다.


In [ ]:
train_generator = train_datagen.flow_from_directory(directory=train_path, target_size=(280, 280), class_mode='binary', batch_size=32, shuffle=True)
valid_generator = valid_datagen.flow_from_directory(directory=valid_path, target_size=(280, 280), class_mode='binary', batch_size=32, shuffle=True)

Found 388 images belonging to 2 classes.
Found 96 images belonging to 2 classes.


#### 3) 학습
- **세부요구사항**
    - Conv2D, MaxPooling2D, Flatten, Dense 레이어들을 이용하여 모델을 설계
    - 학습시 train_generator 이용.
    - validation_data = valid_generator 지정
    - Early Stopping 적용
    - 평가시, confusion matrix, accuracy, recall, precision, f1 score 등을 이용하시오.

* 구조 설계

In [ ]:
from tensorflow.keras.applications import Xception
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [ ]:
def create_model(verbose=False):
    input_tensor = Input(shape=(280, 280, 3))
    pretrained_model = Xception(input_tensor=input_tensor, include_top=False, weights='imagenet')
    pretrained_output = pretrained_model.output

    # customize Classifier layer
    x = GlobalAveragePooling2D()(pretrained_output)
    x = Dense(units=2, activation='relu')(x)
    output = Dense(units=1, activation='sigmoid')(x)

    model = Model(inputs=input_tensor, outputs=output)
    if verbose:
        model.summary()
    return model

In [ ]:
# 모델 정의
model_Xception = create_model(verbose=False)
# 모델 compile
model_Xception.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

* 학습
    * EarlyStopping 설정하기
    * 학습 데이터에 train_generator, validation_data=valid_generator 사용

In [ ]:
# es
es = EarlyStopping(monitor='val_loss',
                   min_delta=0,
                   patience=5,
                   verbose=1,
                   restore_best_weights=True)

In [ ]:
# 모델 학습(fit)
train_hist = model_Xception.fit(train_generator, validation_data=valid_generator,
                                batch_size=32, epochs=500, callbacks=[es], verbose=1)

Epoch 1/500
13/13 [==============================] - 33s 3s/step - loss: 0.3645 - accuracy: 0.9639 - val_loss: 0.3651 - val_accuracy: 0.9375
Epoch 2/500
13/13 [==============================] - 28s 2s/step - loss: 0.3542 - accuracy: 0.9897 - val_loss: 0.4042 - val_accuracy: 0.8958
Epoch 3/500
13/13 [==============================] - 28s 2s/step - loss: 0.3657 - accuracy: 0.9794 - val_loss: 0.3855 - val_accuracy: 0.9271
Epoch 4/500
13/13 [==============================] - 28s 2s/step - loss: 0.3596 - accuracy: 0.9871 - val_loss: 0.3620 - val_accuracy: 0.9583
Epoch 5/500
13/13 [==============================] - 28s 2s/step - loss: 0.3550 - accuracy: 0.9845 - val_loss: 0.3918 - val_accuracy: 0.9375
Epoch 6/500
13/13 [==============================] - 29s 2s/step - loss: 0.3475 - accuracy: 0.9974 - val_loss: 0.3735 - val_accuracy: 0.9583
Epoch 7/500
13/13 [==============================] - 28s 2s/step - loss: 0.3703 - accuracy: 0.9820 - val_loss: 0.3884 - val_accuracy: 0.9896
Epoch 8/500
1

In [ ]:
print(f'모델의 레이어 수 : {len(model_Xception.layers)}')

모델의 레이어 수 : 135


In [ ]:
for idx, layer in enumerate(model_Xception.layers) :
  if idx < 100 :
    layer.trainable = False
  else :
    layer.trainable = True

In [ ]:
model_Xception.compile(loss='binary_crossentropy', metrics=['accuracy'],
                       optimizer=keras.optimizers.Adam(learning_rate=0.001))

In [ ]:
s_hist = model_Xception.fit(train_generator, validation_data = valid_generator, batch_size=32, epochs=500, callbacks=[es], verbose=1)

Epoch 1/500
13/13 [==============================] - 37s 2s/step - loss: 0.0485 - accuracy: 0.9923 - val_loss: 0.3704 - val_accuracy: 0.9792
Epoch 2/500
13/13 [==============================] - 27s 2s/step - loss: 0.0200 - accuracy: 0.9974 - val_loss: 0.0562 - val_accuracy: 0.9792
Epoch 3/500
13/13 [==============================] - 26s 2s/step - loss: 0.0527 - accuracy: 0.9974 - val_loss: 0.9006 - val_accuracy: 0.9375
Epoch 4/500
13/13 [==============================] - 26s 2s/step - loss: 0.0089 - accuracy: 0.9974 - val_loss: 0.6511 - val_accuracy: 0.9375
Epoch 5/500
13/13 [==============================] - 26s 2s/step - loss: 0.0166 - accuracy: 0.9974 - val_loss: 0.1327 - val_accuracy: 0.9792
Epoch 6/500
13/13 [==============================] - 26s 2s/step - loss: 0.0471 - accuracy: 0.9923 - val_loss: 0.2263 - val_accuracy: 0.9896
Epoch 7/500
13/13 [==============================] - 26s 2s/step - loss: 0.0345 - accuracy: 0.9897 - val_loss: 0.0073 - val_accuracy: 0.9896
Epoch 8/500
1

In [ ]:
for idxx, layer in enumerate(model_Xception.layers) :
  if idxx < 110 :
    layer.trainable = False
  else :
    layer.trainable = True

In [ ]:
Acc=[0.9792, 0.9375]

In [ ]:
for idxx, layer in enumerate(model_Xception.layers) :
  if idxx < 130 :
    layer.trainable = False
  else :
    layer.trainable = True

In [ ]:
s_hist = model_Xception.fit(train_generator, validation_data = valid_generator, batch_size=32, epochs=500, callbacks=[es], verbose=1)

Epoch 1/500
13/13 [==============================] - 30s 2s/step - loss: 0.3509 - accuracy: 0.9897 - val_loss: 0.3835 - val_accuracy: 0.9479
Epoch 2/500
13/13 [==============================] - 28s 2s/step - loss: 0.3552 - accuracy: 0.9845 - val_loss: 0.3513 - val_accuracy: 0.9792
Epoch 3/500
13/13 [==============================] - 29s 2s/step - loss: 0.3516 - accuracy: 0.9948 - val_loss: 0.3504 - val_accuracy: 0.9688
Epoch 4/500
13/13 [==============================] - 33s 3s/step - loss: 0.3470 - accuracy: 0.9948 - val_loss: 0.3642 - val_accuracy: 0.9375
Epoch 5/500
13/13 [==============================] - 29s 2s/step - loss: 0.3511 - accuracy: 0.9820 - val_loss: 0.3621 - val_accuracy: 0.9792
Epoch 6/500
13/13 [==============================] - 29s 2s/step - loss: 0.3489 - accuracy: 0.9897 - val_loss: 0.3592 - val_accuracy: 0.9896
Epoch 7/500
13/13 [==============================] - 29s 2s/step - loss: 0.3464 - accuracy: 0.9974 - val_loss: 0.3566 - val_accuracy: 0.9583
Epoch 8/500
1

In [ ]:
0.9792, 0.9375

In [ ]:
s_hist.validation_data

#### 4) 성능 평가
* 평가는 confusion_matrix, classification_report 활용

In [ ]:
y_pred_Xception = model_Xception.predict(x_test)

4/4 [==============================] - 1s 223ms/step


In [ ]:
y_pred_Xception.shape

(121, 1)

In [ ]:
x_test

In [ ]:
performance_test = model_Xception.evaluate(x_test)

4/4 [==============================] - 0s 11ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00


In [ ]:
print('Test Loss : {:.6f},  Test Accuracy : {:.3f}%'.format(performance_test[0], performance_test[1]*100))

Test Loss : 0.000000,  Test Accuracy : 0.000%


### (2) Transfer Learning
- **세부요구사항**
    * VGG16 모델은 1000개의 클래스를 분류하는 데 사용된 ImageNet 데이터셋을 기반으로 사전 학습된 가중치를 가지고 있습니다.
        * 따라서 이 모델은 이미지 분류 문제에 대한 높은 성능을 보입니다.
        * 이 모델은 보통 전이학습(transfer learning)에서 기본적으로 사용되며, 특히 대규모 데이터셋이 없을 때는 기본 모델로 사용되어 fine-tuning을 수행합니다.
    * VGG16 함수로 부터 base_model 저장


In [ ]:
from tensorflow.keras.applications import VGG16

#### 1) VGG16 불러와서 저장하기
* include_top=False로 설정하여 분류기를 제외하고 미리 학습된 가중치 imagenet을 로드합니다.
* .trainable을 True로 설정하여 모델의 모든 레이어들이 fine-tuning에 대해 업데이트되도록 합니다.


In [ ]:
base_model = VGG16(                 )




#### 2) VGG16과 연결한 구조 설계
* VGG16을 불러와서 Flatten, Dense 등으로 레이어 연결하기

#### 3) 학습
- **세부요구사항**
    - 모델 학습 과정에 알맞은 보조 지표를 사용하세요.
    - 데이터
        * Image Generator를 연결하거나
        * 기존 train, validation 셋을 이용해도 됩니다.
        - Early Stopping을 반드시 사용하세요.
        - 최적의 가중치를 모델에 적용하세요.

#### 4) 성능 평가